In [43]:
import csv
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
pathAnn = "/Users/yayafung/Desktop/opp/OPP-115/annotations"
os.chdir(pathAnn)
aDataset = []  # {index: [label, text]}
x = []
y = []
typeIndex = 5
itemIndex = 6
start = len('"selectedText": "')
end = -2


In [44]:
def tfIdf(dataset):
    tfIdfVectorizer = TfidfVectorizer(
        stop_words='english', use_idf=True, ngram_range=(1, 3))
    tfIdf = tfIdfVectorizer.fit_transform(dataset.values())
    df = pd.DataFrame(tfIdf[0].T.todense(
    ), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return df


def readCsv(file_path):
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        label = 0
        for row in csv_reader:
            if (row[5].strip() == "Other"):
                label = 1
            else:
                label = 0
            text = re.findall(
                f'"selectedText"\:(.*?)"\,', row[6])
            if len(text) > 0:
             #   print((row[4] == "0"), label, text)
                for t in text:
                    aDataset.append([label, t])
                    y.append(label)
                    x.append(t)


In [45]:
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".csv"):
        file_path = f"{pathAnn}/{file}"
        readCsv(file_path)


In [46]:
df = pd.DataFrame(aDataset, columns=['label', 'text'])
# print(df)
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=0)

text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(
    1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)
# print(y_test)


In [47]:
X_train[0:10]


[' "When you use a third-party authentication service or link your Company account with a third-party account, you will be asked to provide account information associated with that third-party account.',
 ' "Not selected',
 ' "our third-party advertiser',
 ' "Not selected',
 ' "personal information.',
 ' "These web beacons may be used to deliver cookies that conform to our cookie policy above.',
 ' "aggregate data',
 ' "unless you choose to provide that information to us.',
 ' "Other Information',
 ' "when you view or click on their advertising']

In [48]:
posIndex = []
for i in range(len(y_train)):
    if y_train[i] == 1:
        posIndex.append(i)

posIndex

[5,
 19,
 27,
 83,
 95,
 201,
 216,
 230,
 267,
 298,
 308,
 314,
 323,
 426,
 495,
 501,
 547,
 572,
 598,
 605,
 620,
 625,
 631,
 690,
 708,
 720,
 776,
 807,
 820,
 843,
 848,
 931,
 940,
 950,
 982,
 1014,
 1053,
 1076,
 1081,
 1126,
 1132,
 1167,
 1186,
 1188,
 1192,
 1218,
 1243,
 1259,
 1287,
 1308,
 1330,
 1334,
 1404,
 1440,
 1465,
 1497,
 1511,
 1523,
 1531,
 1557,
 1602,
 1608,
 1666,
 1704,
 1734,
 1792,
 1799,
 1843,
 1901,
 1957,
 2010,
 2022,
 2047,
 2057,
 2067,
 2078,
 2082,
 2145,
 2233,
 2267,
 2294,
 2351,
 2367,
 2446,
 2456,
 2496,
 2502,
 2522,
 2543,
 2584,
 2589,
 2632,
 2648,
 2732,
 2758,
 2827,
 2838,
 2840,
 2852,
 2863,
 2876,
 2902,
 2906,
 3004,
 3018,
 3032,
 3063,
 3114,
 3118,
 3135,
 3154,
 3173,
 3216,
 3236,
 3239,
 3256,
 3258,
 3317,
 3325,
 3339,
 3347,
 3353,
 3361,
 3437,
 3463,
 3506,
 3540,
 3554,
 3563,
 3587,
 3636,
 3678,
 3780,
 3858,
 3861,
 3913,
 3920,
 3950,
 3953,
 4031,
 4082,
 4101,
 4139,
 4153,
 4213,
 4223,
 4236,
 4245,
 4276

In [49]:
len(posIndex)
posSen = []
for i in posIndex:
    posSen.append(X_train[i])

posSen[0:50]

[' "These web beacons may be used to deliver cookies that conform to our cookie policy above.',
 ' "please e-mail us at businessaffairs@lynda.com',
 ' "Also please be aware that when you post messages on our discussion boards, people can respond to your posts through our \\"Reply to Author\\" email function. Of course, they still will not know your email address (unless you send email to them).',
 ' "Some of the documents on the server may contain live references (or \\"pointers\\") to information created and maintained by other organizations. The National Archives does not control and cannot guarantee the relevance, timeliness, or accuracy of the outside materials.',
 ' "In the event that you provide personal information to any third party via any TEN website or any offer made available to you by TEN, you understand that TEN is not responsible for such third parties use or misuse of your personal information. You should review the privacy and information sharing practices of such thir

In [50]:
logit = LogisticRegression(
    C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4, max_iter=400)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
cv_results = cross_val_score(
    logit, X_train_text, y_train, cv=skf, scoring='f1_micro')
print(cv_results, cv_results.mean())
logit.fit(X_train_text, y_train)
test_preds = logit.predict(X_test_text)


[0.9807942  0.98247133 0.98306644 0.98214576 0.98328193] 0.9823519306177267


In [51]:
count = 0
for i in test_preds:
    if i == 1:
        count += 1
print(count)
print(len(test_preds))


273
10269


In [52]:
count = 0
for i in y_test:
    if i == 1:
        count += 1
print(count)
print(len(test_preds))


348
10269


In [53]:
count = 0
for i in range(len(y_test)):
    if y_test[i] == test_preds[i]:
        count += 1
print(len(y_test)-count)


173


In [54]:
X_train_text


<92418x60426 sparse matrix of type '<class 'numpy.float64'>'
	with 780978 stored elements in Compressed Sparse Row format>

In [55]:
pathPolicy = "/Users/yayafung/Desktop/opp/scraped_policies"
os.chdir(pathPolicy)
lines = []
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        print(file)
        file_path = f"{pathPolicy}/{file}"
        with open(file_path) as f:
            lines = lines + f.readlines()


adarshurs.android.vlcmobileremote.txt
a2z.Mobile.Event6243.txt


In [56]:
len(lines)

287

In [57]:
v_lines = text_transformer.transform(lines)
preds = logit.predict(v_lines)

In [58]:
for u in range(len(y_test)):
    if y_test[u] == 1:
        print(u)

9
32
40
56
57
73
79
133
161
206
280
454
512
513
542
571
590
639
643
644
678
726
737
738
772
870
891
896
911
944
1028
1063
1087
1224
1296
1300
1303
1342
1387
1428
1458
1487
1488
1584
1602
1609
1618
1626
1686
1715
1745
1753
1755
1784
1801
1803
1806
1853
1910
1930
1935
1945
1970
2004
2021
2052
2070
2090
2108
2125
2159
2163
2214
2241
2306
2317
2335
2346
2369
2412
2425
2490
2541
2560
2594
2731
2733
2749
2795
2805
2820
2838
2895
2897
2907
2920
2923
2926
2935
2943
2981
3000
3004
3041
3062
3117
3126
3137
3142
3153
3169
3172
3174
3179
3208
3209
3230
3331
3338
3374
3379
3395
3414
3449
3495
3503
3551
3570
3610
3645
3660
3676
3862
3886
3948
3957
3978
4012
4041
4123
4128
4158
4182
4191
4222
4242
4260
4263
4315
4353
4356
4440
4470
4502
4510
4635
4791
4798
4802
4804
4835
4881
4929
4955
4959
4961
4982
4985
5046
5096
5112
5117
5136
5155
5192
5197
5251
5263
5294
5321
5335
5344
5346
5364
5392
5408
5414
5419
5432
5482
5518
5527
5537
5551
5553
5563
5573
5596
5694
5698
5739
5790
5850
5853
5870
5872
5982
611

In [59]:
len(preds)

287

In [60]:
preds

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [65]:
def read_txt(file):
    with open(file) as f:
        text = f.read()
    with open(file) as f:
        lines = f.readlines()
    return text, lines


def remove_other(file):
    text, lines = read_txt(file)
    v_lines = text_transformer.transform(lines)
    print(len(lines))
    preds = logit.predict(v_lines)
    print(preds)
    other_line_index = []
    for p in range(len(preds)):
        if preds[p] == 1:
            other_line_index.append(p)
            print(lines[p])
            text.replace(lines[p], "")
    print("Other:", len(other_line_index))
    print("All:", len(lines))
    return text


remove_other(
    "/Users/yayafung/Desktop/other_classifier_opp/scraped_policies/a2z.Mobile.Event6243.txt")

    



200
[0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
PRIVACY POLICY

Diversified Communications (“Diversified”), together with its affiliates, which includes subsidiaries and entities that Diversified owns globally (collectively, “Diversified,” “we,” or “us”) connect, educate and strengthen business communities through market-leading events, digital products and publications. This Privacy Policy (“Policy”) describes our general privacy policy and practices in relation to Diversified’s websites, events, and services (collectively, the “Services”).

How we collect and store information depends on the websites you visit, the activities i

'Skip to content\n  \nHome\nMenu\nPRIVACY POLICY\n\nDiversified Communications (“Diversified”), together with its affiliates, which includes subsidiaries and entities that Diversified owns globally (collectively, “Diversified,” “we,” or “us”) connect, educate and strengthen business communities through market-leading events, digital products and publications. This Privacy Policy (“Policy”) describes our general privacy policy and practices in relation to Diversified’s websites, events, and services (collectively, the “Services”).\n\nIn compliance with both the General Data Protection Regulation (EU) 2016/679 of the European Parliament and of the Council of 27 April 2016 (“GDPR”) and The California Consumer Privacy Act of 2018 (“CPPA), this policy serves as Diversified’s GDPR Data Collection and Processing Notice and the CCPA required Notice.\n\n1. Applicability & Unique Rights for Some Persons\n\nThis Policy applies to all individuals who provide personal data and personal information 